In [2]:
import numpy as np 
import pandas as pd 

df_train=pd.read_csv(r"D:\GeakMinds Internship\real world projects\datasets\store train.csv")
df_test=pd.read_csv(r"D:\GeakMinds Internship\real world projects\datasets\store test.csv")
df_info=pd.read_csv(r"D:\GeakMinds Internship\real world projects\datasets\sample_submission.csv")

In [3]:
df_train

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [4]:
df_test

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
...,...,...,...,...
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50


In [5]:
df_info

,id,sales
0,0,52
1,1,52
2,2,52
3,3,52
4,4,52
...,...,...
44995,44995,52
44996,44996,52
44997,44997,52
44998,44998,52


In [6]:
df_test=df_test.merge(df_info,on="id")


In [7]:
df_train.groupby(["store", "item"]).agg({"sales": ["sum", "mean", "median", "std"]})

sales                             
               sum       mean median        std
store item                                     
1     1      36468  19.971522   19.0   6.741022
      2      97050  53.148959   52.0  15.005779
      3      60638  33.208105   33.0  10.072529
      4      36440  19.956188   20.0   6.640618
      5      30335  16.612815   16.0   5.672102
...            ...        ...    ...        ...
10    46    120601  66.046550   65.0  18.114991
      47     45204  24.755750   24.0   7.924820
      48    105570  57.814896   57.0  15.898538
      49     60317  33.032311   32.0  10.091610
      50    135192  74.037240   73.0  19.937566

[500 rows x 4 columns]

In [8]:
df = pd.concat([df_train, df_test], sort=False)

In [9]:
def create_date_features(df, date_column):
    df['month'] = df[date_column].dt.month
    df['day_of_month'] = df[date_column].dt.day
    df['day_of_year'] = df[date_column].dt.dayofyear
    df['week_of_year'] = df[date_column].dt.isocalendar().week.astype(int)
    df['day_of_week'] = df[date_column].dt.dayofweek
    df['year'] = df[date_column].dt.year
    df["is_wknd"] = (df[date_column].dt.weekday // 4).astype(int)
    df['is_month_start'] =df[date_column].dt.is_month_start.astype(int)
    df['is_month_end'] = df[date_column].dt.is_month_end.astype(int)
    df['quarter'] = df[date_column].dt.quarter
    df['is_quarter_start'] = df[date_column].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df[date_column].dt.is_quarter_end.astype(int)
    df['is_year_start'] = df[date_column].dt.is_year_start.astype(int)
    df['is_year_end'] = df[date_column].dt.is_year_end.astype(int)
    return df

In [10]:
df.date = pd.to_datetime(df.date)
df = create_date_features(df, "date")

In [11]:
def random_noise(dataframe):
    return np.random.normal(scale=1.6, size=(len(dataframe),))

def lag_features(dataframe, lags):
    for lag in lags:
        dataframe['sales_lag_' + str(lag)] = dataframe.groupby(["store", "item"])["sales"].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe)
    return dataframe

In [12]:
df = lag_features(df, [91,92,170,171,172,173,174,175,176,177,178,179,
                       180,181,182,183,184,185,186,187,188,189,190,
                       350,351,352,352,354,355,356,357,358,359,360,
                       361,362,363,364,365,366,367,368,369,370,538,
                       539,540,541,542,718,719,720,721,722])

In [13]:
df

,date,store,item,sales,id,month,day_of_month,day_of_year,week_of_year,day_of_week,...,sales_lag_538,sales_lag_539,sales_lag_540,sales_lag_541,sales_lag_542,sales_lag_718,sales_lag_719,sales_lag_720,sales_lag_721,sales_lag_722
0,2013-01-01,1,1,13,NaN,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,1,1,11,NaN,1,2,2,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,1,1,14,NaN,1,3,3,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,1,1,13,NaN,1,4,4,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,1,1,10,NaN,1,5,5,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,2018-03-27,10,50,52,44995.0,3,27,86,13,1,...,68.970114,60.640082,78.678911,100.695926,98.332599,90.757834,82.562097,69.624784,71.532885,53.230141
44996,2018-03-28,10,50,52,44996.0,3,28,87,13,2,...,87.732191,67.213694,61.785340,79.049624,101.698111,84.968136,89.920758,81.596272,65.548065,70.008785
44997,2018-03-29,10,50,52,44997.0,3,29,88,13,3,...,91.264056,84.633428,70.442720,61.366939,79.542932,101.696397,86.117636,91.325550,84.699925,68.593228
44998,2018-03-30,10,50,52,44998.0,3,30,89,13,4,...,83.540395,91.083566,84.514741,67.360692,59.350092,55.130734,100.257662,84.140688,88.339956,82.735955


exponentially weighted moving

In [14]:
def ewm_features(dataframe, alphas, lags):
    new_columns = {}
    for alpha in alphas:
        for lag in lags:
            new_column_name = f'sales_ewm_alpha_{str(alpha).replace(".", "")}_lag_{lag}'
            new_columns[new_column_name] = dataframe.groupby(["store", "item"])["sales"].transform(
                lambda x: x.shift(lag).ewm(alpha=alpha).mean()
            )
    return pd.concat([dataframe, pd.DataFrame(new_columns)], axis=1)

In [15]:
alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [91,92,178,179,180,181,182,359,360,361,449,450,451,539,540,541,629,630,631,720]
df = ewm_features(df, alphas, lags)

In [23]:
df.shape

(958000, 174)

In [16]:
df = pd.get_dummies(df, columns=['store', 'item', 'day_of_week', 'month'])
df['sales'] = np.log1p(df["sales"].values)

In [17]:
# Training set will include all data before 2017
train = df.loc[(df["date"] < "2017-01-01"), :]

# Validation set will include the first 3 months of 2017
val = df.loc[(df["date"] >= "2017-01-01") & (df["date"] < "2017-04-01"), :]

In [18]:
cols = [col for col in train.columns if col not in ["date", "id", "sales", "year"]]

Y_train = train["sales"]
X_train = train[cols]

Y_val = val["sales"]
X_val = val[cols]

Y_train.shape, X_train.shape, Y_val.shape, X_val.shape

((730500,), (730500, 243), (45000,), (45000, 243))

In [19]:
from matplotlib import pyplot as plt
import seaborn as sns
import lightgbm as lgb
import warnings

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

# Define the model's hyperparameters in a dictionary
lgb_params = {
    'num_leaves': 10,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'max_depth': 5,
    'verbose': 0,
    'num_boost_round': 1000,
    'nthread': -1
}

In [ ]:
def smape(preds, target):    #Calculates the Symmetric Mean Absolute Percentage Error (SMAPE).

    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val

def lgbm_smape(preds, train_data):

    labels = train_data.get_label()
    #  back from log-scale to original scale
    smape_val = smape(np.expm1(preds), np.expm1(labels))

    return 'SMAPE', smape_val, False

In [21]:
lgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)
lgbval = lgb.Dataset(data=X_val, label=Y_val, reference=lgbtrain, feature_name=cols)


model = lgb.train(lgb_params,
                  lgbtrain,
                  valid_sets=[lgbtrain, lgbval],
                  num_boost_round=lgb_params['num_boost_round'],
                  callbacks=[
                      lgb.early_stopping(stopping_rounds=200),
                      lgb.log_evaluation(period=100)
                  ],
                  feval=lgbm_smape)

y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)

smape(np.expm1(y_pred_val), np.expm1(Y_val))

Training until validation scores don't improve for 200 rounds
[100]	training's l2: 0.0654993	training's SMAPE: 19.1293	valid_1's l2: 0.0522489	valid_1's SMAPE: 17.2986
[200]	training's l2: 0.0484246	training's SMAPE: 16.3736	valid_1's l2: 0.038298	valid_1's SMAPE: 15.1536
[300]	training's l2: 0.0432324	training's SMAPE: 15.6643	valid_1's l2: 0.0352865	valid_1's SMAPE: 14.6628
[400]	training's l2: 0.0397818	training's SMAPE: 15.258	valid_1's l2: 0.0346005	valid_1's SMAPE: 14.5631
[500]	training's l2: 0.037461	training's SMAPE: 14.9473	valid_1's l2: 0.0340728	valid_1's SMAPE: 14.4593
[600]	training's l2: 0.0357922	training's SMAPE: 14.6969	valid_1's l2: 0.0337616	valid_1's SMAPE: 14.3888
[700]	training's l2: 0.0346075	training's SMAPE: 14.504	valid_1's l2: 0.0334399	valid_1's SMAPE: 14.317
[800]	training's l2: 0.0336768	training's SMAPE: 14.3454	valid_1's l2: 0.0331764	valid_1's SMAPE: 14.2566
[900]	training's l2: 0.0329509	training's SMAPE: 14.2115	valid_1's l2: 0.0328894	valid_1's SMAP

14.146435896730747

In [22]:
def plot_lgb_importances(model, plot=False, num=10):

    gain = model.feature_importance('gain')
    feat_imp = pd.DataFrame({
        'feature': model.feature_name(),
        'split': model.feature_importance('split'),
        'gain': 100 * gain / gain.sum()
    }).sort_values('gain', ascending=False)

    if plot:
        plt.figure(figsize=(10, 10))
        sns.set(font_scale=1)
        sns.barplot(x="gain", y="feature", data=feat_imp[0:25])
        plt.title('Feature Importance')
        plt.tight_layout()
        plt.show()
    else:
        print(feat_imp.head(num))
    
    return feat_imp

In [23]:
feat_imp = plot_lgb_importances(model, num=264)

                         feature  split       gain
144    sales_ewm_alpha_05_lag_91    440  34.625555
124    sales_ewm_alpha_07_lag_91    201  13.490547
47                 sales_lag_364    450  12.390628
40                 sales_lag_357    203   7.732522
34                 sales_lag_350    238   6.398935
..                           ...    ...        ...
142   sales_ewm_alpha_07_lag_631      0   0.000000
139   sales_ewm_alpha_07_lag_541      0   0.000000
138   sales_ewm_alpha_07_lag_540      0   0.000000
136   sales_ewm_alpha_07_lag_451      0   0.000000
67   sales_ewm_alpha_095_lag_179      0   0.000000

[243 rows x 3 columns]


In [25]:
importance_zero = feat_imp[feat_imp["gain"] == 0]["feature"].values
len(importance_zero)

68

In [26]:
imp_feats = [col for col in cols if col not in importance_zero]
len(imp_feats)

175

In [34]:
imp_feats

['day_of_month',
 'day_of_year',
 'week_of_year',
 'is_wknd',
 'quarter',
 'sales_lag_91',
 'sales_lag_92',
 'sales_lag_170',
 'sales_lag_171',
 'sales_lag_172',
 'sales_lag_173',
 'sales_lag_174',
 'sales_lag_175',
 'sales_lag_176',
 'sales_lag_177',
 'sales_lag_178',
 'sales_lag_179',
 'sales_lag_180',
 'sales_lag_181',
 'sales_lag_182',
 'sales_lag_183',
 'sales_lag_184',
 'sales_lag_185',
 'sales_lag_186',
 'sales_lag_187',
 'sales_lag_188',
 'sales_lag_189',
 'sales_lag_190',
 'sales_lag_350',
 'sales_lag_351',
 'sales_lag_352',
 'sales_lag_354',
 'sales_lag_355',
 'sales_lag_356',
 'sales_lag_357',
 'sales_lag_358',
 'sales_lag_361',
 'sales_lag_362',
 'sales_lag_363',
 'sales_lag_364',
 'sales_lag_365',
 'sales_lag_366',
 'sales_lag_367',
 'sales_lag_368',
 'sales_lag_369',
 'sales_lag_370',
 'sales_lag_538',
 'sales_lag_539',
 'sales_lag_542',
 'sales_lag_718',
 'sales_lag_719',
 'sales_lag_721',
 'sales_lag_722',
 'sales_ewm_alpha_095_lag_91',
 'sales_ewm_alpha_095_lag_92',
 '

In [35]:
test

,date,sales,id,day_of_month,day_of_year,week_of_year,year,is_wknd,is_month_start,is_month_end,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12


In [40]:
df_test.sales

0        52
1        52
2        52
3        52
4        52
         ..
44995    52
44996    52
44997    52
44998    52
44999    52
Name: sales, Length: 45000, dtype: int64

In [ ]:
train = df.loc[~df.sales.isna()]
Y_train = train['sales']
X_train = train[imp_feats]


test = df.loc[df_test.sales]
X_test = test[imp_feats]

lgb_params = {'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'nthread': -1,
              "num_boost_round": model.best_iteration}

In [30]:
lgbtrain_all = lgb.Dataset(data=X_train, label=Y_train, feature_name=imp_feats)

final_model = lgb.train(lgb_params, lgbtrain_all, num_boost_round=model.best_iteration)



In [31]:
final_model

In [42]:
test_preds = final_model.predict(X_test, num_iteration=model.best_iteration)

In [45]:
test_preds[0]

2.8913946351897284

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


y_pred = final_model.predict(X_train)

mae = mean_absolute_error(Y_train, y_pred)


rmse = np.sqrt(mean_squared_error(Y_train, y_pred))


r2 = r2_score(Y_train, y_pred)

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")


MAE: 0.1494
RMSE: 0.2033
R²: 0.8709
